# Limpeza e preparação de dados (Parte-1)
## Base de nascidos vivos do DataSUS
O DataSUS disponibiliza diversos arquivos de dados com relação a seus segurados, conforme a [lei da transparência de informações públicas](https://www.sisgov.com/transparencia-acesso-informacao/#:~:text=A%20Lei%20da%20Transpar%C3%AAncia%20(LC,em%20um%20site%20na%20internet.). 

Essas informações podem ser obtidas pela internet [aqui](http://www2.datasus.gov.br/DATASUS/index.php?area=0901&item=1). Como o processo de obtenção desses arquivos foge um pouco do nosso escopo, deixamos o arquivo SINASC_RO_2019.csv` já como vai ser encontrado no DataSUS. O dicionário de dados está no arquivo `estrutura_sinasc_para_CD.pdf` (o nome do arquivo tal qual no portal do DataSUS).

### Nosso objetivo
Queremos deixar uma base organizada para podermos estudar a relação entre partos com risco para o bebê e algumas condições como tempo de parto, consultas de pré-natal etc.

#### Preparação da base
1. Carregue a base 'SINASC_RO_2019.csv'. Conte o número de registros e o número de registros não duplicados da base. Dica: você aprendeu um método que remove duplicados, encadeie este método com um outro método que conta o número de linhas. **Há linhas duplicadas?**  

2. Conte o número de valores *missing* por variável.  

3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.  

4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.  

5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.<br>
6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.  
7. Das restantes, decida que valore te parece mais adequado (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.  
8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:
- Entre 8 e 10 está em uma faixa 'normal'. 
- Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
- Entre 4 e 5 significa 'asfixia moderada'.
- Entre 0 e 3 significa 'asfixia severa'.  

Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.  
<br>
9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.

In [3]:
# Importando as bibliotecas
import pandas as pd
import numpy as np

In [5]:
# 1) 
sinasc = pd.read_csv('SINASC_RO_2019.csv')
# Explorando o DataFrame
sinasc.head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,...,KOTELCHUCK,CONTADOR,munResStatus,munResTipo,munResNome,munResUf,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,8 a 11 anos,NaN,0.0,0.0,...,5,1,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,2,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,8 a 11 anos,513205.0,2.0,0.0,...,5,3,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,12 anos ou mais,231205.0,0.0,0.0,...,4,4,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,8 a 11 anos,999992.0,1.0,0.0,...,5,5,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia,-11.93554,-61.99982,338.0,7067.025


In [7]:
# Shape mostra o número de linhas e colunas
# Cada linha corresponde a um registro
print(sinasc.shape) 
# Desconsiderando os dados duplicados com:
sinasc.drop_duplicates().shape
# O DataFrame inicial e final tem a mesma dimensão, ou seja,
# Não há dados duplicados

(27028, 69)


(27028, 69)

In [9]:
# Colunas com dados categóricos do DataFrame
sinasc.select_dtypes('object').head()

,ESCMAE,GESTACAO,GRAVIDEZ,PARTO,DTNASC,SEXO,RACACOR,DTCADASTRO,CODANOMAL,VERSAOSIST,DTRECEBIM,munResStatus,munResTipo,munResNome,munResUf
0,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-19,Masculino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
1,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-21,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
2,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-02-25,Feminino,Branca,2019-03-08,NaN,3.2.01,2019-03-08,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia
3,12 anos ou mais,37 a 41 semanas,Única,Cesáreo,2019-03-20,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alto Alegre dos Parecis,Rondônia
4,8 a 11 anos,37 a 41 semanas,Única,Cesáreo,2019-03-23,Feminino,Parda,2019-04-03,NaN,3.2.01,2019-04-09,ATIVO,MUNIC,Alta Floresta D'Oeste,Rondônia


In [11]:
# Colunas com dados numéricos do DataFrame
sinasc.select_dtypes('number').head()

,ORIGEM,CODESTAB,CODMUNNASC,LOCNASC,IDADEMAE,ESTCIVMAE,CODOCUPMAE,QTDFILVIVO,QTDFILMORT,CODMUNRES,...,STDNNOVA,CODPAISRES,TPROBSON,PARIDADE,KOTELCHUCK,CONTADOR,munResLat,munResLon,munResAlt,munResArea
0,1,2679477.0,110001,1,19,5.0,NaN,0.0,0.0,110001,...,1,1,1,0,5,1,-11.93554,-61.99982,338.0,7067.025
1,1,2679477.0,110001,1,29,2.0,999992.0,1.0,0.0,110001,...,1,1,5,1,5,2,-11.93554,-61.99982,338.0,7067.025
2,1,2679477.0,110001,1,37,9.0,513205.0,2.0,0.0,110001,...,1,1,7,1,5,3,-11.93554,-61.99982,338.0,7067.025
3,1,2516500.0,110001,1,30,5.0,231205.0,0.0,0.0,110037,...,1,1,2,0,4,4,-12.13178,-61.85308,397.0,3958.273
4,1,2516500.0,110001,1,30,2.0,999992.0,1.0,0.0,110001,...,1,1,5,1,5,5,-11.93554,-61.99982,338.0,7067.025


> **2. Conte o número de valores *missing* por variável.**

In [14]:
# A quantidade de valores faltantes pode ser obtida da seguinte maneira
sinasc.isna().sum()

ORIGEM          0
CODESTAB      115
CODMUNNASC      0
LOCNASC         0
IDADEMAE        0
             ... 
munResUf        0
munResLat       1
munResLon       1
munResAlt       1
munResArea      1
Length: 69, dtype: int64

 > **3. Ok, no item anterior você deve ter achado pouco prático ler a informação de tantas variáveis, muitas delas nem devem ser interesantes. Então crie uma seleção dessa base somente com as colunas que interessam. São elas:
` 
['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 
    'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5'] 
`
Refaça a contagem de valores *missings*.**  

In [17]:
# 3) 
colunas_interesse = ['LOCNASC', 'IDADEMAE', 'ESTCIVMAE', 'ESCMAE', 'QTDFILVIVO', 'GESTACAO', 'GRAVIDEZ', 'CONSULTAS', 'APGAR5']
# Criando um novo DataFrame com o conteúdo de interesse
df = sinasc[colunas_interesse]
df.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


In [19]:
# Obtendo a quantidade de valores missings 
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

> **4. Apgar é uma *nota* que o pediatra dá ao bebê quando nasce de acordo com algumas características associadas principalmente à respiração. Apgar 1 e Apgar 5 são as notas 1 e 5 minutos do nascimento. Apgar5 será a nossa variável de interesse principal. Então remova todos os registros com Apgar5 não preenchido. Para esta seleção, conte novamente o número de linhas e o número de *missings*.**

In [22]:
# 4) Variável de interesse Apgar5
print(df['APGAR5'].shape)
df.isna().sum()

(27028,)


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      317
ESCMAE         312
QTDFILVIVO    1573
GESTACAO      1232
GRAVIDEZ        79
CONSULTAS        0
APGAR5         103
dtype: int64

In [24]:
# Utilizando o método .dropna(subset=[]) para removar os dados faltantes 
# com relação a coluna APGAR5
df = df.dropna(subset=['APGAR5'])

In [26]:
# Não há mais dados faltantes na variável de interesse
print('O numero de linhas foi reduzido para {}.'.format(df.shape[0]))
print('Ou seja, as 103 linhas de dados faltantes de APGAR5 foram removidas.')
df.isna().sum()

O numero de linhas foi reduzido para 26925.
Ou seja, as 103 linhas de dados faltantes de APGAR5 foram removidas.


LOCNASC          0
IDADEMAE         0
ESTCIVMAE      315
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

> **5. observe que as variáveis `['ESTCIVMAE', 'CONSULTAS']` possuem o código `9`, que significa *ignorado*. Vamos assumir que o não preenchido é o mesmo que o código `9`.**

In [30]:
# 5.
# Valores faltantes em 'ESTCIVMAE'
print(df.ESTCIVMAE.isna().sum())
# Valores de código 9 em 'ESTCIVMAE'
df[(df['ESTCIVMAE']) == 9].shape

315


(144, 9)

In [32]:
# Não há valores faltantes em 'CONSULTAS'
print(df.CONSULTAS.isna().sum())
# Valores de código 9 em 'CONSULTAS'
print(df[(df['CONSULTAS']) == 9].shape)

0
(50, 9)


In [34]:
# Tornando os não preenchidos (missing) em código 9
# Fazendo uso do método .fillna()

df['ESTCIVMAE'] = df['ESTCIVMAE'].fillna(9)
# Mesmo CONSULTAS não possuindo missing, pode-se fazer o mesmo 
df['CONSULTAS'] = df['CONSULTAS'].fillna(9)

# Não há mais valores faltantes nessas colunas
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO    1566
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

In [36]:
# Verificando novamente a quantidade do código 9
print(df[df['ESTCIVMAE'] == 9].shape)
print(df[df['CONSULTAS'] == 9].shape)

(459, 9)
(50, 9)


> **6. Substitua os valores faltantes da quantitativa (`QTDFILVIVO`) por zero.**

In [40]:
# 6) 
df['QTDFILVIVO'] = df['QTDFILVIVO'].fillna(0)
# Dessa forma o número de missings na coluna agora é zero
df.isna().sum()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64

> **7. Das restantes, decida que valores parecem mais adequados (um 'não preenchido' ou um valor 'mais provável' como no item anterior) e preencha. Justifique. Lembre-se de que tratamento de dados é trabalho do cientista, e que estamos tomando decisões a todo o momento - não há necessariamente certo e errado aqui.**

In [44]:
# 7) 
print(df.isna().sum())

# Todas são variáveis categóricas
df[['ESCMAE', 'GESTACAO', 'GRAVIDEZ']].describe().transpose()

LOCNASC          0
IDADEMAE         0
ESTCIVMAE        0
ESCMAE         310
QTDFILVIVO       0
GESTACAO      1216
GRAVIDEZ        76
CONSULTAS        0
APGAR5           0
dtype: int64


,count,unique,top,freq
ESCMAE,26615,5,8 a 11 anos,15581
GESTACAO,25709,6,37 a 41 semanas,22523
GRAVIDEZ,26849,3,Única,26356


In [46]:
# Avaliando os dados referentes a escolaridade da mãe
print('--> A quantidade de dados faltantes em "ESCMAE" em relação ao df como um todo é: {}%'
      .format((df['ESCMAE'].isna().sum() / df.shape[0]) * 100))

escolaridade = df['ESCMAE'].value_counts()
escolaridade = pd.DataFrame(escolaridade)
escolaridade['%'] = (df['ESCMAE'].value_counts()/df.shape[0]) * 100
escolaridade


--> A quantidade de dados faltantes em "ESCMAE" em relação ao df como um todo é: 1.1513463324048283%


,count,%
ESCMAE,,
8 a 11 anos,15581,57.868152
12 anos ou mais,5757,21.381616
4 a 7 anos,4727,17.556175
1 a 3 anos,510,1.894150
Nenhuma,40,0.148561


In [48]:
# Avaliando os dados da variável gestação
print('--> A quantidade de dados faltantes em "GESTACAO" em relação ao df como um todo é: {}%'
      .format((df['GESTACAO'].isna().sum() / df.shape[0]) * 100))

gestacao = df['GESTACAO'].value_counts()
gestacao = pd.DataFrame(gestacao)
gestacao['%'] = (df['GESTACAO'].value_counts()/df.shape[0]) * 100
gestacao

--> A quantidade de dados faltantes em "GESTACAO" em relação ao df como um todo é: 4.516248839368616%


,count,%
GESTACAO,,
37 a 41 semanas,22523,83.650882
32 a 36 semanas,2234,8.297122
42 semanas e mais,698,2.592386
28 a 31 semanas,162,0.601671
22 a 27 semanas,85,0.315692
Menos de 22 semanas,7,0.025998


In [50]:
# Analisando os dados associados a gravidez
print('--> A quantidade de dados faltantes em "GRAVIDEZ" em relação ao df como um todo é: {}%'
      .format((df['GRAVIDEZ'].isna().sum() / df.shape[0]) * 100))

gravidez = df['GRAVIDEZ'].value_counts()
gravidez = pd.DataFrame(gravidez)
gravidez['%'] = (df['GRAVIDEZ'].value_counts()/df.shape[0]) * 100
gravidez

--> A quantidade de dados faltantes em "GRAVIDEZ" em relação ao df como um todo é: 0.2822655524605385%


,count,%
GRAVIDEZ,,
Única,26356,97.886722
Dupla,488,1.812442
Tríplice e mais,5,0.018570


Para decidir qual decisão tomar da forma com a qual tratar esse dados, pode ser interessante analisar as proporções entre o DataFrame 'raw' (cru) e entre o DataFrame sem os dados faltantes (removendo-os).

In [53]:
# Removendo os dados faltantes (em um novo DataFrame)
df_sem_faltantes = df.dropna()
print(df_sem_faltantes.shape)
print(df.shape)

# Relação entre os DataFrames
proporcao = (df_sem_faltantes.shape[0] / df.shape[0]) * 100
print('--> Os dados tratados, considerando a remoção dos faltantes, correpondem a {}% dos dados do DataFrame'.format(round(proporcao,2)))
df_sem_faltantes.head()

(25375, 9)
(26925, 9)
--> Os dados tratados, considerando a remoção dos faltantes, correpondem a 94.24% dos dados do DataFrame


,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0


Em um primeiro momento, no que tange a abordagem de tais variáveis,  consideraria a remoção dos dados **faltantes** para a coluna `ESCMAE`, uma vez que esses são uma pequena proporção do DataFrame como um todo. Por outro lado, para as colunas  `GRAVIDEZ` e `GESTACAO` parece ser plausível substituir os dados **faltantes** por "Única" e “37 a 41 semanas” já que esses correspondem a ~ 98% e 84% dos dados associados à gestação e à gravidez, respectivamente. Porém, ainda parece ser uma solução mais interessante descartar todos esses dados faltantes (para as três variáveis) e trabalhar com o DataFrame limpo.

> **8. O Apgar possui uma classificação indicando se o bebê passou por asfixia:**
> - Entre 8 e 10 está em uma faixa 'normal'. 
> - Entre 6 e 7, significa que o recém-nascido passou por 'asfixia leve'. 
> - Entre 4 e 5 significa 'asfixia moderada'.
> - Entre 0 e 3 significa 'asfixia severa'.  

> Crie uma categorização dessa variável com essa codificação e calcule as frequências dessa categorização.

In [58]:
# 8) 
df.loc[(df['APGAR5'] >= 8) & (df['APGAR5'] <= 10), 'apgar5_cat'] = 'normal'
df.loc[(df['APGAR5'] >= 6) & (df['APGAR5'] <= 7), 'apgar5_cat'] = 'asfixia_leve'
df.loc[(df['APGAR5'] >= 4) & (df['APGAR5'] <= 5), 'apgar5_cat'] = 'asfixia_moderada'
df.loc[(df['APGAR5'] >= 0) & (df['APGAR5'] <= 3), 'apgar5_cat'] = 'asfixia_severa'
df.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,apgar5_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [60]:
# Criando um DataFrame com as ocorrências
ocorrencias = df['apgar5_cat'].value_counts()
ocorrencias = pd.DataFrame(ocorrencias)
ocorrencias['%'] = (df['apgar5_cat'].value_counts()/df.shape[0]) * 100
ocorrencias

,count,%
apgar5_cat,,
normal,26463,98.284123
asfixia_leve,320,1.188487
asfixia_severa,74,0.274838
asfixia_moderada,68,0.252553


In [64]:
# Fazendo o mesmo para o DataFrame com os dados faltantes removidos
df_sem_faltantes.loc[(df_sem_faltantes['APGAR5'] >= 8) & (df_sem_faltantes['APGAR5'] <= 10), 'apgar5_cat'] = 'normal'
df_sem_faltantes.loc[(df_sem_faltantes['APGAR5'] >= 6) & (df_sem_faltantes['APGAR5'] <= 7), 'apgar5_cat'] = 'asfixia_leve'
df_sem_faltantes.loc[(df_sem_faltantes['APGAR5'] >= 4) & (df_sem_faltantes['APGAR5'] <= 5), 'apgar5_cat'] = 'asfixia_moderada'
df_sem_faltantes.loc[(df_sem_faltantes['APGAR5'] >= 0) & (df_sem_faltantes['APGAR5'] <= 3), 'apgar5_cat'] = 'asfixia_severa'
df_sem_faltantes.head()

,LOCNASC,IDADEMAE,ESTCIVMAE,ESCMAE,QTDFILVIVO,GESTACAO,GRAVIDEZ,CONSULTAS,APGAR5,apgar5_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal


In [70]:
ocorrencias_sem_faltantes = df_sem_faltantes['apgar5_cat'].value_counts()
ocorrencias_sem_faltantes = pd.DataFrame(ocorrencias_sem_faltantes)
ocorrencias_sem_faltantes['%'] = (df_sem_faltantes['apgar5_cat'].value_counts()/df_sem_faltantes.shape[0]) * 100
ocorrencias_sem_faltantes

# Nesse DF as proporções se mantêm em relação ao geral
# porém uma variação mais notável emerge para a variável 'asfixia_moderada'
# Dessa forma, remover os dados faltantes continua sendo uma opção interessante

,count,%
apgar5_cat,,
normal,24946,98.309360
asfixia_leve,302,1.190148
asfixia_severa,68,0.267980
asfixia_moderada,59,0.232512


> **9. Renomeie as variáveis para que fiquem no *snake case*, ou seja, em letras minúsculas, com um *underscore* entre as palávras. Dica: repare que se você não quiser criar um *dataframe* novo, você vai precisar usar a opção `inplace = True`.**

In [73]:
# 9) 
df.columns = ['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo', 
              'gestacao', 'gravidez', 'consultas', 'apgar_5', 'apgar5_cat']
print(df.columns)
df.head()

Index(['loc_nasc', 'idade_mae', 'est_civ_mae', 'esc_mae', 'qtd_fil_vivo',
       'gestacao', 'gravidez', 'consultas', 'apgar_5', 'apgar5_cat'],
      dtype='object')


,loc_nasc,idade_mae,est_civ_mae,esc_mae,qtd_fil_vivo,gestacao,gravidez,consultas,apgar_5,apgar5_cat
0,1,19,5.0,8 a 11 anos,0.0,37 a 41 semanas,Única,4,10.0,normal
1,1,29,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,9.0,normal
2,1,37,9.0,8 a 11 anos,2.0,37 a 41 semanas,Única,4,10.0,normal
3,1,30,5.0,12 anos ou mais,0.0,37 a 41 semanas,Única,3,10.0,normal
4,1,30,2.0,8 a 11 anos,1.0,37 a 41 semanas,Única,4,10.0,normal
